# Problem

There is a possibility to pay a credit card invoice next month

# Solution

Standardization dataset, apply K neighbors nearest model and output predicted file with probability to 1 (Default payment (1=yes, 0=no))

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

### Import dataset

In [ ]:
df_train = pd.read_csv('../input/av-janata-hack-payment-default-prediction/train_20D8GL3.csv')
df_test = pd.read_csv('../input/av-janata-hack-payment-default-prediction/test_O6kKpvt.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

### Drop Columns

In [ ]:
df_train.drop(['SEX','EDUCATION','MARRIAGE'], axis=1, inplace=True)
df_test.drop(['SEX','EDUCATION','MARRIAGE'], axis=1, inplace=True)

### Describe Variabels

In [ ]:
df_train.drop(['ID', 'default_payment_next_month'], axis=1).describe()

### Correlations

In [ ]:
df_train.corr().style.background_gradient()

### Standardization

In [ ]:
scaler = StandardScaler()
aux = df_train.loc[:, df_train.columns[9:-1]]
aux['LIMIT_BAL'] = df_train['LIMIT_BAL']
x = scaler.fit_transform(aux)

In [ ]:
df_train.loc[:, df_train.columns[9:-1]] = x[:,0:-1]
df_train['LIMIT_BAL'] = x[:,-1]
df_train.head()

### Split dataset in training and test

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(df_train.loc[:, df_train.columns[1:-1]].values, df_train.loc[:, df_train.columns[-1]].values, test_size=0.3, random_state=42)

### KNN

In [ ]:
score = []
for value in np.arange(1, 50):
    knn = KNeighborsClassifier(n_neighbors=value)
    knn.fit(X_train,Y_train)
    score.append(knn.score(x_test, y_test))

### Score

In [ ]:
plt.plot([index for index in np.arange(len(score))], score)
plt.scatter([index for index in np.arange(len(score))], score)
plt.show()

In [ ]:
print('N = {}'.format(score.index(max(score))))
print('Score = {}'.format(max(score)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=score.index(max(score)))
knn.fit(X_train,Y_train)

### Predict dataset test

In [ ]:
aux = df_test.loc[:, df_test.columns[9:]]
aux['LIMIT_BAL'] = df_test['LIMIT_BAL']
x = scaler.fit_transform(aux)
df_test.loc[:, df_test.columns[9:]] = x[:,0:-1]
df_test['LIMIT_BAL'] = x[:,-1]

In [ ]:
predict = knn.predict_proba(df_test.loc[:, df_test.columns[1:]].values)

In [ ]:
predict = pd.DataFrame(predict)

In [ ]:
df_test['default_payment_next_month'] = predict[1]

In [ ]:
df_test = df_test[['ID', 'default_payment_next_month']]

### Output dataset

In [ ]:
df_test.to_csv('output.csv', index=False, header=True)